In [ ]:
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
bos_token='</s>', eos_token='</s>', unk_token='<unk>',
pad_token='<pad>', mask_token='<mask>')
tokenizer.tokenize("안녕하세요. 한국어 GPT-2 입니다.😤:)l^o")

In [ ]:
import torch
from transformers import GPT2LMHeadModel

model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')
text = '근육이 커지기 위해서는'
input_ids = tokenizer.encode(text, return_tensors='pt')
gen_ids = model.generate(input_ids,
                           max_length=128,
                           repetition_penalty=2.0,
                           pad_token_id=tokenizer.pad_token_id,
                           eos_token_id=tokenizer.eos_token_id,
                           bos_token_id=tokenizer.bos_token_id,
                           use_cache=True)
generated = tokenizer.decode(gen_ids[0])
print(generated)

In [ ]:
text = '''
유구한 역사와 전통에 빛나는 우리 대한국민은 3·1운동으로 건립된 대한민국임시정부의 법통과 불의에 항거한 4·19민주이념을 계승하고,
조국의 민주개혁과 평화적 통일의 사명에 입각하여 정의·인도와 동포애로써 민족의 단결을 공고히 하고,
모든 사회적 폐습과 불의를 타파하며, 자율과 조화를 바탕으로 자유민주적 기본질서를 더욱 확고히 하여
정치·경제·사회·문화의 모든 영역에 있어서 각인의 기회를 균등히 하고,
능력을 최고도로 발휘하게 하며, 자유와 권리에 따르는 책임과 의무를 완수하게 하여,
안으로는 국민생활의 균등한 향상을 기하고 밖으로는 항구적인 세계평화와 인류공영에 이바지함으로써
우리들과 우리들의 자손의 안전과 자유와 행복을 영원히 확보할 것을 다짐하면서
1948년 7월 12일에 제정되고 8차에 걸쳐 개정된 헌법을 이제 국회의 의결을 거쳐 국민투표에 의하여 개정한다.
'''

In [ ]:
# 토크나이져 모델을 로드
import torch
from transformers import GPT2LMHeadModel
from transformers import PreTrainedTokenizerFast

tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
bos_token='</s>', eos_token='</s>', unk_token='<unk>',
pad_token='<pad>', mask_token='<mask>')

model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')
model.eval()
# 프롬프트 엔지니어링
prompt = f'{text}\n\한줄 요약:'
# 인코딩
input_ids = tokenizer.encode(prompt, return_tensors='pt')
# 모델 추론(생성)
with torch.no_grad():
  gen_ids = model.generate(input_ids,
                          #  max_length=128,
                           max_new_tokens = 100,
                           repetition_penalty=2.0, # 같은 말을 반복하지 않도록 벌점
                           pad_token_id=tokenizer.pad_token_id,
                           eos_token_id=tokenizer.eos_token_id,
                           bos_token_id=tokenizer.bos_token_id,
                           use_cache=True,
                           do_sample=True, # 자연스러운 문장
                           temperature=0.7, # 창의성 조정(낮을수록 보수적, 높을수록 환각가능성 높음)
                           top_k=50, # 확률 적으로 상위 50개 단어중에서 선택
                           )
  generated = tokenizer.decode(gen_ids[0],skip_special_tokens=True)
  # 프롬프트 이후에 생성된 텍스트만 추출
  summary = generated[len(prompt):].strip()
  print(summary)

